In [2]:
!git clone https://github.com/GDPlumb/MAPLE.git

Cloning into 'MAPLE'...
remote: Enumerating objects: 93, done.
remote: Total 93 (delta 0), reused 0 (delta 0), pack-reused 93
Unpacking objects: 100% (93/93), done.


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from google.colab import files
import pandas as pd
import json
import re
import ast
from MAPLE.Code import MAPLE
import numpy as np

In [4]:
def process_spectra(file_obj, file_name):
    file_string = str(file_obj[file_name])

    return file_string.split("\\n")

In [57]:
def process_matrix(file_obj, file_name):
    file_string = str(file_obj[file_name])

    result = file_string.split("\\n")
    result[0] = result[0].replace("b'", "")

    for i in range(len(result)):
        result[i] = result[i].split(" ")
        result[i] = [int(x) if x.isdigit() else x for x in result[i]]
        lastel = result[i][-1]
        if lastel == "+":
            result[i][-1] = 1
        elif lastel == "-":
            result[i][-1] = 0
    
    return result

In [88]:
def write_output_file(filename, obj, is_reverse=False):
    with open(filename, 'w') as file:
        sorted_dict = sorted(obj.items(), key=lambda x: x[1], reverse=is_reverse)
        for i in sorted_dict:
            file.write(";".join([i[0], str(i[1])]) + "\n")

In [59]:
spectra_file = files.upload()
matrix_file = files.upload()

Saving spectra to spectra (2)


Saving matrix to matrix (3)


In [60]:
spectra_list = process_spectra(spectra_file, list(spectra_file.keys())[0])
spectra_list[-1] = "Pass/Fail"
spectra_list[0] = spectra_list[0].replace("b'", "")

In [61]:
matrix_list = process_matrix(matrix_file, list(matrix_file.keys())[0])

In [62]:
matrix_list.pop()

["'"]

In [63]:
df = pd.DataFrame(data=matrix_list, columns=spectra_list)

In [64]:
df

,org.apache.commons.math3.util.MathArrays$1#674,org.apache.commons.math3.util.MathArrays$1#678,org.apache.commons.math3.util.MathArrays$1#680,org.apache.commons.math3.util.MathArrays$1#681,org.apache.commons.math3.util.MathArrays$1#683,org.apache.commons.math3.util.MathArrays$1#684,org.apache.commons.math3.util.MathArrays$1#687,org.apache.commons.math3.util.MathArrays$1#689,org.apache.commons.math3.util.MathArrays#51,org.apache.commons.math3.util.MathArrays#88,org.apache.commons.math3.util.MathArrays#89,org.apache.commons.math3.util.MathArrays#92,org.apache.commons.math3.util.MathArrays#93,org.apache.commons.math3.util.MathArrays#94,org.apache.commons.math3.util.MathArrays#96,org.apache.commons.math3.util.MathArrays#110,org.apache.commons.math3.util.MathArrays#111,org.apache.commons.math3.util.MathArrays#114,org.apache.commons.math3.util.MathArrays#115,org.apache.commons.math3.util.MathArrays#116,org.apache.commons.math3.util.MathArrays#118,org.apache.commons.math3.util.MathArrays#132,org.apache.commons.math3.util.MathArrays#133,org.apache.commons.math3.util.MathArrays#136,org.apache.commons.math3.util.MathArrays#137,org.apache.commons.math3.util.MathArrays#138,org.apache.commons.math3.util.MathArrays#140,org.apache.commons.math3.util.MathArrays#154,org.apache.commons.math3.util.MathArrays#155,org.apache.commons.math3.util.MathArrays#158,org.apache.commons.math3.util.MathArrays#159,org.apache.commons.math3.util.MathArrays#160,org.apache.commons.math3.util.MathArrays#162,org.apache.commons.math3.util.MathArrays#173,org.apache.commons.math3.util.MathArrays#174,org.apache.commons.math3.util.MathArrays#175,org.apache.commons.math3.util.MathArrays#177,org.apache.commons.math3.util.MathArrays#188,org.apache.commons.math3.util.MathArrays#189,org.apache.commons.math3.util.MathArrays#190,...,org.apache.commons.math3.random.RandomDataImpl#127,org.apache.commons.math3.random.RandomDataImpl#128,org.apache.commons.math3.random.RandomDataImpl#136,org.apache.commons.math3.random.RandomDataImpl#156,org.apache.commons.math3.random.RandomDataImpl#161,org.apache.commons.math3.random.RandomDataImpl#166,org.apache.commons.math3.random.RandomDataImpl#186,org.apache.commons.math3.random.RandomDataImpl#191,org.apache.commons.math3.random.RandomDataImpl#196,org.apache.commons.math3.random.RandomDataImpl#213,org.apache.commons.math3.random.RandomDataImpl#218,org.apache.commons.math3.random.RandomDataImpl#233,org.apache.commons.math3.random.RandomDataImpl#248,org.apache.commons.math3.random.RandomDataImpl#265,org.apache.commons.math3.random.RandomDataImpl#279,org.apache.commons.math3.random.RandomDataImpl#293,org.apache.commons.math3.random.RandomDataImpl#307,org.apache.commons.math3.random.RandomDataImpl#320,org.apache.commons.math3.random.RandomDataImpl#336,org.apache.commons.math3.random.RandomDataImpl#363,org.apache.commons.math3.random.RandomDataImpl#383,org.apache.commons.math3.random.RandomDataImpl#401,org.apache.commons.math3.random.RandomDataImpl#415,org.apache.commons.math3.random.RandomDataImpl#431,org.apache.commons.math3.random.RandomDataImpl#447,org.apache.commons.math3.random.RandomDataImpl#461,org.apache.commons.math3.random.RandomDataImpl#462,org.apache.commons.math3.random.RandomDataImpl#472,org.apache.commons.math3.random.RandomDataImpl#473,org.apache.commons.math3.random.RandomDataImpl#485,org.apache.commons.math3.random.RandomDataImpl#486,org.apache.commons.math3.random.RandomDataImpl#493,org.apache.commons.math3.random.RandomDataImpl#494,org.apache.commons.math3.random.RandomDataImpl#518,org.apache.commons.math3.random.RandomDataImpl#519,org.apache.commons.math3.random.RandomDataImpl#532,org.apache.commons.math3.random.RandomDataImpl#550,org.apache.commons.math3.random.RandomDataImpl#565,org.apache.commons.math3.random.RandomDataImpl#581,Pass/Fail
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0

In [65]:
x = df[spectra_list[:len(spectra_list)-1]]
y = df["Pass/Fail"]

In [67]:
fails = []
passes = []
i = 0
for result in y:
    if result == 0:
        fails.append(i)
    else:
        passes.append(i)
    i += 1 

In [68]:
if len(fails) == 1:
    df = df.append(df.iloc[fails[0]])
    x = df[spectra_list[:len(spectra_list)-1]]
    y = df["Pass/Fail"]

In [69]:
if (df.shape[0] < 6):
    df = df.append(df)
    x = df[spectra_list[:len(spectra_list)-1]]
    y = df["Pass/Fail"]

In [70]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=1)

In [71]:
model = RandomForestClassifier(n_estimators=100, bootstrap=True, max_features='sqrt', random_state=1)

In [72]:
model.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [73]:
exp = MAPLE.MAPLE(np.array(x_train), np.array(model.predict(x_train)), np.array(x_test), np.array(model.predict(x_test)))

In [75]:
feature_coefficients_based_on_error_instance = {}

for i in fails:
    cur_coefs = list(exp.explain(np.array(x.iloc[i]))['coefs'])
    feature_coefficients_based_on_error_instance[i] = []
    for index, val in enumerate(cur_coefs[:-1]):
        test_string = spectra_list[index] + ";" + str(val)
        feature_coefficients_based_on_error_instance[i].append(test_string) 

In [83]:
coef_values_on_predict = {}

for i in feature_coefficients_based_on_error_instance:
    for j in feature_coefficients_based_on_error_instance[i]:
        feature_and_value = j.split(";")
        value = float(feature_and_value[1])
        feature = feature_and_value[0]
        if feature not in coef_values_on_predict:
            coef_values_on_predict[feature] = []
        coef_values_on_predict[feature].append(value)

In [94]:
feature_coef_means = {}
feature_coef_max = {}
feature_coef_min = {}

for i in coef_values_on_predict:
    feautre_coef_means[i] = sum(coef_values_on_predict[i]) / len(coef_values_on_predict[i])
    feature_coef_max[i] = max(coef_values_on_predict[i])
    feature_coef_min[i] = min(coef_values_on_predict[i]) 

In [87]:
with open("Math-8_maple_results.txt", "w") as file:
    for i in feature_coefficients_based_on_error_instance:
        file.write(str(i) + "\n")
        for j in range(len(feature_coefficients_based_on_error_instance[i])):
            file.write(feature_coefficients_based_on_error_instance[i][j] + "\n")

In [96]:
write_output_file('Math-8_maple_mean.txt', feautre_coef_means)

In [98]:
write_output_file('Math-8_maple_max.txt', feature_coef_max)
write_output_file('Math-8_maple_min.txt', feature_coef_min)